In [2]:
!pip install Korpora

     |████████████████████████████████| 57 kB 2.6 MB/s 
     |████████████████████████████████| 96 kB 3.4 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [3]:
# 챗봅 라이브러리 
from Korpora import KoreanChatbotKorpus
corpus = KoreanChatbotKorpus()


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : songys@github
    Repository : https://github.com/songys/Chatbot_data
    References :

    Chatbot_data_for_Korean v1.0
      1. 챗봇 트레이닝용 문답 페어 11,876개
      2. 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링
    자세한 내용은 위의 repository를 참고하세요.

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[korean_chatbot_data] download ChatbotData.csv: 893kB [00:00, 9.34MB/s]


In [4]:
print(corpus.get_all_texts()[:5])
print(corpus.get_all_pairs()[:5])

['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
('하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.', '여행은 언제나 좋죠.', '눈살이 찌푸려지죠.')


In [5]:
print('Q:', corpus.train[0].text)
print('A:', corpus.train[0].pair)

Q: 12시 땡!
A: 하루가 또 가네요.


In [6]:
# 데이터 전처리
len(corpus.get_all_texts())

11823

In [7]:
# 2000개 셋만 활용
texts = []
pairs = []
for i, (text, pair) in enumerate(zip(corpus.get_all_texts(), corpus.get_all_pairs())):
  texts.append(text)
  pairs.append(pair)
  if i >= 2000:
    break

In [8]:
list(zip(texts, pairs))[1995:2000]

[('배 아프다', '약이 필요하면 도움을 받아보세요.'),
 ('배 터지겠네', '위를 좀 쉬게 해주세요.'),
 ('배 터지겠다.', '산책 좀 해야겠네여.'),
 ('배가 너무 고파', '뭐 좀 챙겨드세요.'),
 ('배가 넘넘 고파', '저도 밥 먹고 싶어요')]

In [11]:
import re
def clean_sentence(sentence):
  sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣]', r'', sentence)
  return sentence

In [12]:
print(clean_sentence('안녕하세요~~~~:)'))
print(clean_sentence('텐서플로^@^#@!'))

안녕하세요
텐서플로


In [13]:
# 형태소분석기
!pip install konlpy

     |████████████████████████████████| 19.4 MB 5.0 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 448 kB 69.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [19]:
from konlpy.tag import Okt
okt = Okt()
def process_morph(sentence):
  return ' '.join(okt.morphs(sentence))

In [15]:
# 문장처리
def clean_and_morph(sentence, is_question=True):
  # 한글 문장 전처리
  sentence = clean_sentence(sentence)
  # 형태소
  sentence = process_morph(sentence)
  # Question인 경우 Answer인 경우를 분기 처리
  if is_question:
    return sentence
  else:
    # START 토큰은 decoder input에 END 토큰은 decoder output에 추가.
    return ('<START> ' + sentence, sentence + ' <END>')


In [21]:
def preprocess(texts, pairs):
  questions = []
  answer_in = []
  answer_out = []
  for text in texts:
    question = clean_and_morph(text, is_question=True)
    questions.append(question)
  for pair in pairs:
    in_, out_ = clean_and_morph(pair, is_question=False)
    answer_in.append(in_)
    answer_out.append(out_)

  return questions, answer_in, answer_out

In [22]:
questions, answer_in, answer_out = preprocess(texts, pairs)

In [24]:
print(questions[:2])
print(answer_in[:2])
print(answer_out[:2])

['12시 땡', '1 지망 학교 떨어졌어']
['<START> 하루 가 또 가네요', '<START> 위로 해드립니다']
['하루 가 또 가네요 <END>', '위로 해드립니다 <END>']


In [25]:
# 전체 문장을 하나의 리스트로
all_sentences = questions + answer_in + answer_out

In [26]:
# 토큰화
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
tokenizer.fit_on_texts(all_sentences)

In [28]:
# 단어사전 확인
for word, idx in tokenizer.word_index.items():
  print(f'{word} \t -> \t{idx}')
  if idx > 10:
    break

<OOV> 	 -> 	1
<START> 	 -> 	2
<END> 	 -> 	3
이 	 -> 	4
을 	 -> 	5
가 	 -> 	6
요 	 -> 	7
해보세요 	 -> 	8
는 	 -> 	9
사람 	 -> 	10
보세요 	 -> 	11


In [29]:
len(tokenizer.word_index)

4103

In [30]:
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

In [31]:
# 문장 길이 맞추기
MAX_LENGTH = 30
question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
answer_in_padded = pad_sequences(answer_in_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
answer_out_padded = pad_sequences(answer_out_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')

In [32]:
question_padded.shape, answer_in_padded.shape, answer_out_padded.shape

((2001, 30), (2001, 30), (2001, 30))

In [33]:
# 모델
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [34]:
# 인코더 (학습용 Encoder)
class Encoder(Model):
  def __init__(self, units, vocab_size, embedding_dim, time_steps):
    super(Encoder, self).__init__()
    self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps)
    self.dropout = Dropout(0.2)
    self.lstm = LSTM(units, return_state=True)
  def call(self, inputs):
    x = self.embedding(inputs)
    x = self.dropout(x)
    x, hidden_state, cell_state = self.lstm(x)
    return [hidden_state, cell_state]

In [46]:
# 디코더 (학습용 Decoder)
class Decoder(Model):
  def __init__(self, units, vocab_size, embedding_dim, time_steps):
    super(Decoder, self).__init__()
    self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps)
    self.dropout = Dropout(0.2)
    self.lstm = LSTM(units, return_state=True, return_sequences=True)
    self.dense = Dense(vocab_size, activation='softmax')
  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    x = self.dropout(x)
    x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
    x = self.dense(x)
    return x, hidden_state, cell_state

In [51]:
# 모델 결합
class Seq2Seq(Model):
  def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
    super(Seq2Seq, self).__init__()
    self.start_token = start_token
    self.end_token = end_token
    self.time_steps = time_steps
    self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
    self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)
  def call(self, inputs, training=True):
    if training:
      encoder_inputs, decoder_inputs = inputs
      context_vector = self.encoder(encoder_inputs)
      decoder_outputs, _, _ = self.decoder(inputs=decoder_inputs, initial_state=context_vector)
      return decoder_outputs
    else:
      context_vector = self.encoder(inputs)
      target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
      results = tf.TensorArray(tf.int32, self.time_steps)
      for i in tf.range(self.time_steps):
        decoder_output, decoder_hidden, decoder_cell = self.decoder(target_seq, initial_state=context_vector)
        decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), dtype=tf.int32)
        decoder_output = tf.reshape(decoder_output, shape=(1, 1))
        result = results.write(i, decoder_output)
        if decoder_output == self.end_token:
          break
        target_seq = decoder_output
        context_vector = [decoder_hidden, decoder_cell]
      return tf.reshape(results.stack(), shape=(1, self.time_steps))


In [37]:
# 단어별 원핫인코딩
VOCAB_SIZE = len(tokenizer.word_index)+1

In [39]:
def convert_to_one_hot(padded):
  one_hot_vector = np.zeros((len(answer_out_padded), MAX_LENGTH, VOCAB_SIZE))
  for i, sequence in enumerate(answer_out_padded):
    for j, index in enumerate(sequence):
      one_hot_vector[i, j, index] = 1
  return one_hot_vector

In [40]:
answer_in_one_hot = convert_to_one_hot(answer_in_padded)
answer_out_one_hot = convert_to_one_hot(answer_out_padded)
answer_in_one_hot[0].shape, answer_out_one_hot[0].shape

((30, 4104), (30, 4104))

In [41]:
# 변환된 index를 다시 단어로 변환 함수
def convert_index_to_text(indexs, end_token):
  sentence = ''
  # 모든 문장 반복
  for index in indexs:
    if index == end_token:
      break
    if index > 0 and tokenizer.index_word[index] is not None:
      sentence += tokenizer.index_word[index]
    else:
      sentence += ''
    sentence += ' '
  return sentence

In [42]:
# 학습
BUFFER_SIZER = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = MAX_LENGTH
START_TOKEN = tokenizer.word_index['<START>']
END_TOKEN = tokenizer.word_index['<END>']
UNITS = 128
VOCAB_SIZE = len(tokenizer.word_index)+1
DATA_LENGTH = len(questions)
SMAPLE_SIZE = 3
NUM_EPOCHS = 20

In [44]:
checkpoint_path = 'model/seq2seq-chatbot-no-attention-checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, save_best_only=True,
                             monitor='loss', verbose=1)

In [48]:
seq2seq = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)
seq2seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [49]:
def make_prediction(model, question_inputs):
  results = model(inputs=question_inputs, training=False)
  results = np.asarray(results).reshape(-1)
  return results

In [ ]:
for epoch in range(NUM_EPOCHS):
  print(f'processing epoch: {epoch * 10 + 1}...')
  seq2seq.fit([question_padded, answer_in_padded],
              answer_out_one_hot,
              epochs=10,
              batch_size=BATCH_SIZE,
              callbacks=[checkpoint])
  # 예측 성능 검증
  samples = np.random.randint(DATA_LENGTH, size=SMAPLE_SIZE)
  for idx in samples:
    question_inputs = question_padded[idx]
    results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))
    results = convert_index_to_text(results, END_TOKEN)
    print(f'Q: {question[idx]}')
    print(f'A: {results}\n')
    print()


processing epoch: 1...
Epoch 1/10
126/126 [==============================] - 19s 148ms/step - loss: 0.8570 - acc: 0.8615

Epoch 00001: loss improved from 0.87148 to 0.85703, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
Epoch 2/10
126/126 [==============================] - 19s 147ms/step - loss: 0.8418 - acc: 0.8623

Epoch 00002: loss improved from 0.85703 to 0.84181, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
Epoch 3/10
126/126 [==============================] - 19s 148ms/step - loss: 0.8279 - acc: 0.8630

Epoch 00003: loss improved from 0.84181 to 0.82787, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
Epoch 4/10
126/126 [==============================] - 19s 148ms/step - loss: 0.8130 - acc: 0.8638

Epoch 00004: loss improved from 0.82787 to 0.81301, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
Epoch 5/10
126/126 [==============================] - 19s 147ms/step - loss: 0.7980 - acc: 0.8645

Epoch 